In [ ]:
import polars as pl
import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Question 3

df1 = pl.read_excel("data/timeinvar-1.xlsx")
df2 = pl.read_excel("data/timevar-1.xlsx")
df = df1.join(df2, on="id", how="left",validate="1:m")
df = df.with_columns(
    hs=pl.when(pl.col("edu") <= 12).then(1).otherwise(0),
    col=pl.when((pl.col("edu") > 12) & (pl.col("edu") <= 16)).then(1).otherwise(0),
    tao1=pl.when((pl.col("edu") <= 16)).then(pl.col("edu")).otherwise(0),
    tao2=pl.when((pl.col("edu") > 16)).then(pl.col("edu")).otherwise(0),
    grad=pl.when(pl.col("edu") > 16).then(1).otherwise(0),
    eduab=pl.col("edu") * pl.col("ability"),
    edu2=pl.col("edu")**2,
)
data = df.to_pandas()
df.columns

In [ ]:
formula  = "lwage ~ edu + ability + exper + meduc + feduc + brokenhome + siblings"


model = smf.ols(formula=formula, data=data).fit()
print(model.summary())

In [ ]:
formula  = "lwage ~ col + grad + ability + exper + meduc + feduc + brokenhome + siblings"


model = smf.ols(formula=formula, data=data).fit()
print(model.summary())

In [ ]:
# b

formula  = "lwage ~ edu + edu2 + ability + exper + meduc + feduc + brokenhome + siblings"


model = smf.ols(formula=formula, data=data).fit()
print(model.summary())

In [ ]:
# c

formula  = "lwage ~ edu + eduab + ability + exper + meduc + feduc + brokenhome + siblings"

model = smf.ols(formula=formula, data=data).fit(cov_type='HC1')

print(model.summary())

In [ ]:
data
plt.scatter(data["edu"], data["lwage"])
plt.show()

In [ ]:
coeffs = model.params[['edu', 'educab']].values
cov = model.cov_params().loc[['edu', 'educab'], ['edu', 'educab']].values

delta_x = np.array([1, 0.052374]) 

delta_ln_assaults = np.dot(delta_x, coeffs)

std_error = np.sqrt(np.dot(delta_x, np.dot(cov, delta_x)))

lower = delta_ln_assaults - 1.96 * std_error
upper = delta_ln_assaults + 1.96 * std_error

change = (np.exp(delta_ln_assaults) - 1)
ci_lower = (np.exp(lower) - 1)
ci_upper = (np.exp(upper) - 1)

print(f"Predicted change in assaults: {change:.2f}")
print(f"95% CI: [{ci_lower:.2f}, {ci_upper:.2f}]")

In [ ]:
#d
# INFO: Missign the second part of (d)

formula  = "lwage ~ edu + edu2 + educab + ability + exper + meduc + feduc + brokenhome + siblings"

model = smf.ols(formula=formula, data=data).fit(cov_type='HC1')

print(model.summary())

In [ ]:
formula  = "lwage ~ tao1 + tao2 + ability + exper + meduc + feduc + brokenhome + siblings"


model = smf.ols(formula=formula, data=data).fit()
print(model.summary())